In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_hdf('data/train_preprocessed_v4.h5')
train_df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance,JFK_distance,invalid
0,7.5,-73.978463,40.759072,-73.963577,40.774255,1,2014.0,9,6,15,1.719640,21.029044,0
1,6.5,-73.982397,40.770245,-73.972450,40.764062,1,2012.0,5,1,7,1.122217,21.940741,0
2,12.9,-74.006507,40.718808,-73.961096,40.760711,1,2009.0,5,2,20,5.210553,20.679895,0
3,20.1,-73.999242,40.734165,-73.972597,40.751228,2,2010.0,7,4,22,3.008668,21.891888,0
4,4.1,-73.977750,40.746274,-73.972760,40.755768,1,2010.0,8,2,13,0.626738,21.931951,0


In [3]:
test_df = pd.read_hdf('data/test_prepro_v4.h5')
test_df.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance,JFK_distance,invalid,key
0,-73.972484,40.742743,-73.918937,40.764496,1,2009.0,1,4,0,5.991612,16.117319,0,2009-01-01 00:01:04.0000003
1,-73.985850,40.722826,-73.986301,40.739347,1,2009.0,1,4,0,0.509264,23.237424,0,2009-01-01 00:01:26.0000001
2,-73.988917,40.740142,-73.982769,40.777291,1,2009.0,1,4,0,1.328903,23.142153,0,2009-01-01 00:04:42.0000001
3,-73.977163,40.764490,-73.914474,40.771575,1,2009.0,1,4,0,6.974103,15.690509,0,2009-01-01 00:04:54.0000001
4,-73.948849,40.778003,-73.977678,40.748692,2,2009.0,1,4,0,3.329684,19.450750,0,2009-01-01 00:04:59.0000004


In [18]:
y = train_df['fare_amount']
X = train_df.drop(columns = ['fare_amount'])
X.head()

,index,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance,JFK_distance,invalid
0,22123337,-74.001794,40.730871,-73.993615,40.733085,1,2014.0,11,3,15,0.911992,24.131007,0
1,26445322,-73.776687,40.645387,-73.932267,40.641768,5,2011.0,10,1,18,17.300069,0.202003,0
2,3735797,-74.008812,40.719458,-73.989417,40.758688,4,2013.0,11,3,20,2.469191,23.775087,0
3,41189573,-73.992443,40.724385,-74.001607,40.740810,1,2010.0,11,6,14,1.136601,23.971398,0
4,9637071,-73.981668,40.761493,-73.980391,40.759927,1,2010.0,11,5,18,0.149906,22.789952,0


In [7]:
train_df = train_df.sample(frac = 0.20)
train_df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance,JFK_distance,invalid
22123337,5.0,-74.001794,40.730871,-73.993615,40.733085,1,2014.0,11,3,15,0.911992,24.131007,0
26445322,27.7,-73.776687,40.645387,-73.932267,40.641768,5,2011.0,10,1,18,17.300069,0.202003,0
3735797,12.5,-74.008812,40.719458,-73.989417,40.758688,4,2013.0,11,3,20,2.469191,23.775087,0
41189573,16.1,-73.992443,40.724385,-74.001607,40.740810,1,2010.0,11,6,14,1.136601,23.971398,0
9637071,4.1,-73.981668,40.761493,-73.980391,40.759927,1,2010.0,11,5,18,0.149906,22.789952,0


In [17]:
train_df.reset_index(inplace = True)

In [9]:
import lightgbm as lgb

params = {
        'learning_rate': 0.65,
        'application': 'regression',
        'max_depth': 3,
        'num_leaves': 150,
        'verbosity': -1,
        'metric': 'RMSE',
    }

In [20]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=10, random_state=None) 
iter_no = 0
for train_index, test_index in kf.split(X):
    iter_no += 1
    print ("Iteration Number = "+ str(iter_no))
    X_train, X_test = X.loc[train_index, :], X.loc[test_index, :] 
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    gbm = lgb.train(params,
                    lgb_train,
                    valid_sets=lgb_eval,
                    num_boost_round=300,
                    early_stopping_rounds = 10)
    gbm.save_model('model_kfold_v' + str(iter_no) + '.txt')

Iteration Number = 1
[1]	valid_0's rmse: 6.14639
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 5.31045
[3]	valid_0's rmse: 4.9946
[4]	valid_0's rmse: 4.79859
[5]	valid_0's rmse: 4.73127
[6]	valid_0's rmse: 4.66139
[7]	valid_0's rmse: 4.61532
[8]	valid_0's rmse: 4.58667
[9]	valid_0's rmse: 4.56377
[10]	valid_0's rmse: 4.55011
[11]	valid_0's rmse: 4.50342
[12]	valid_0's rmse: 4.46788
[13]	valid_0's rmse: 4.44898
[14]	valid_0's rmse: 4.3733
[15]	valid_0's rmse: 4.36187
[16]	valid_0's rmse: 4.34126
[17]	valid_0's rmse: 4.33062
[18]	valid_0's rmse: 4.32453
[19]	valid_0's rmse: 4.31279
[20]	valid_0's rmse: 4.30571
[21]	valid_0's rmse: 4.26549
[22]	valid_0's rmse: 4.24796
[23]	valid_0's rmse: 4.21465
[24]	valid_0's rmse: 4.19581
[25]	valid_0's rmse: 4.18794
[26]	valid_0's rmse: 4.17705
[27]	valid_0's rmse: 4.16871
[28]	valid_0's rmse: 4.15488
[29]	valid_0's rmse: 4.14694
[30]	valid_0's rmse: 4.13948
[31]	valid_0's rmse: 4.12584
[32]	valid_0's rmse: 4.10536
[

[276]	valid_0's rmse: 3.75283
[277]	valid_0's rmse: 3.75229
[278]	valid_0's rmse: 3.75178
[279]	valid_0's rmse: 3.7517
[280]	valid_0's rmse: 3.75136
[281]	valid_0's rmse: 3.75152
[282]	valid_0's rmse: 3.75117
[283]	valid_0's rmse: 3.75036
[284]	valid_0's rmse: 3.75024
[285]	valid_0's rmse: 3.75004
[286]	valid_0's rmse: 3.74969
[287]	valid_0's rmse: 3.74884
[288]	valid_0's rmse: 3.74867
[289]	valid_0's rmse: 3.74815
[290]	valid_0's rmse: 3.74746
[291]	valid_0's rmse: 3.74678
[292]	valid_0's rmse: 3.74626
[293]	valid_0's rmse: 3.74575
[294]	valid_0's rmse: 3.74507
[295]	valid_0's rmse: 3.74479
[296]	valid_0's rmse: 3.74432
[297]	valid_0's rmse: 3.74357
[298]	valid_0's rmse: 3.74313
[299]	valid_0's rmse: 3.74302
[300]	valid_0's rmse: 3.74249
Did not meet early stopping. Best iteration is:
[300]	valid_0's rmse: 3.74249
Iteration Number = 2
[1]	valid_0's rmse: 6.10769
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 5.2847
[3]	valid_0's rmse: 4.97206
[4]	vali

[249]	valid_0's rmse: 3.76825
[250]	valid_0's rmse: 3.76779
[251]	valid_0's rmse: 3.7676
[252]	valid_0's rmse: 3.76678
[253]	valid_0's rmse: 3.76599
[254]	valid_0's rmse: 3.76548
[255]	valid_0's rmse: 3.76472
[256]	valid_0's rmse: 3.76417
[257]	valid_0's rmse: 3.76358
[258]	valid_0's rmse: 3.76309
[259]	valid_0's rmse: 3.76252
[260]	valid_0's rmse: 3.76216
[261]	valid_0's rmse: 3.76199
[262]	valid_0's rmse: 3.76132
[263]	valid_0's rmse: 3.76126
[264]	valid_0's rmse: 3.7609
[265]	valid_0's rmse: 3.76065
[266]	valid_0's rmse: 3.76038
[267]	valid_0's rmse: 3.75958
[268]	valid_0's rmse: 3.75925
[269]	valid_0's rmse: 3.75883
[270]	valid_0's rmse: 3.75842
[271]	valid_0's rmse: 3.75801
[272]	valid_0's rmse: 3.75751
[273]	valid_0's rmse: 3.75706
[274]	valid_0's rmse: 3.75653
[275]	valid_0's rmse: 3.75597
[276]	valid_0's rmse: 3.75552
[277]	valid_0's rmse: 3.7552
[278]	valid_0's rmse: 3.75495
[279]	valid_0's rmse: 3.75448
[280]	valid_0's rmse: 3.75416
[281]	valid_0's rmse: 3.75426
[282]	valid_0

[222]	valid_0's rmse: 3.78282
[223]	valid_0's rmse: 3.78187
[224]	valid_0's rmse: 3.78113
[225]	valid_0's rmse: 3.78073
[226]	valid_0's rmse: 3.78004
[227]	valid_0's rmse: 3.77926
[228]	valid_0's rmse: 3.77896
[229]	valid_0's rmse: 3.7782
[230]	valid_0's rmse: 3.77773
[231]	valid_0's rmse: 3.77631
[232]	valid_0's rmse: 3.77627
[233]	valid_0's rmse: 3.77626
[234]	valid_0's rmse: 3.77551
[235]	valid_0's rmse: 3.77475
[236]	valid_0's rmse: 3.77444
[237]	valid_0's rmse: 3.77377
[238]	valid_0's rmse: 3.77335
[239]	valid_0's rmse: 3.77285
[240]	valid_0's rmse: 3.77242
[241]	valid_0's rmse: 3.7722
[242]	valid_0's rmse: 3.77202
[243]	valid_0's rmse: 3.77148
[244]	valid_0's rmse: 3.77156
[245]	valid_0's rmse: 3.77141
[246]	valid_0's rmse: 3.77128
[247]	valid_0's rmse: 3.77047
[248]	valid_0's rmse: 3.76986
[249]	valid_0's rmse: 3.76779
[250]	valid_0's rmse: 3.76653
[251]	valid_0's rmse: 3.76634
[252]	valid_0's rmse: 3.76607
[253]	valid_0's rmse: 3.76533
[254]	valid_0's rmse: 3.76487
[255]	valid_

[195]	valid_0's rmse: 3.83357
[196]	valid_0's rmse: 3.83228
[197]	valid_0's rmse: 3.83154
[198]	valid_0's rmse: 3.83133
[199]	valid_0's rmse: 3.83041
[200]	valid_0's rmse: 3.83003
[201]	valid_0's rmse: 3.82932
[202]	valid_0's rmse: 3.82892
[203]	valid_0's rmse: 3.82844
[204]	valid_0's rmse: 3.82785
[205]	valid_0's rmse: 3.82666
[206]	valid_0's rmse: 3.82653
[207]	valid_0's rmse: 3.82588
[208]	valid_0's rmse: 3.82549
[209]	valid_0's rmse: 3.8252
[210]	valid_0's rmse: 3.82443
[211]	valid_0's rmse: 3.82356
[212]	valid_0's rmse: 3.82249
[213]	valid_0's rmse: 3.82229
[214]	valid_0's rmse: 3.82153
[215]	valid_0's rmse: 3.82101
[216]	valid_0's rmse: 3.82054
[217]	valid_0's rmse: 3.81983
[218]	valid_0's rmse: 3.81943
[219]	valid_0's rmse: 3.81873
[220]	valid_0's rmse: 3.81898
[221]	valid_0's rmse: 3.81852
[222]	valid_0's rmse: 3.81803
[223]	valid_0's rmse: 3.81813
[224]	valid_0's rmse: 3.81757
[225]	valid_0's rmse: 3.81705
[226]	valid_0's rmse: 3.81647
[227]	valid_0's rmse: 3.81568
[228]	valid

...

[141]	valid_0's rmse: 3.87625
[142]	valid_0's rmse: 3.87525
[143]	valid_0's rmse: 3.87425
[144]	valid_0's rmse: 3.87354
[145]	valid_0's rmse: 3.87174
[146]	valid_0's rmse: 3.87062
[147]	valid_0's rmse: 3.86978
[148]	valid_0's rmse: 3.86851
[149]	valid_0's rmse: 3.8675
[150]	valid_0's rmse: 3.86712
[151]	valid_0's rmse: 3.86552
[152]	valid_0's rmse: 3.86426
[153]	valid_0's rmse: 3.86258
[154]	valid_0's rmse: 3.86182
[155]	valid_0's rmse: 3.86126
[156]	valid_0's rmse: 3.86072
[157]	valid_0's rmse: 3.85958
[158]	valid_0's rmse: 3.85886
[159]	valid_0's rmse: 3.85725
[160]	valid_0's rmse: 3.85762
[161]	valid_0's rmse: 3.85672
[162]	valid_0's rmse: 3.85494
[163]	valid_0's rmse: 3.85398
[164]	valid_0's rmse: 3.85363
[165]	valid_0's rmse: 3.85275
[166]	valid_0's rmse: 3.85182
[167]	valid_0's rmse: 3.85087
[168]	valid_0's rmse: 3.85002
[169]	valid_0's rmse: 3.84946
[170]	valid_0's rmse: 3.84779
[171]	valid_0's rmse: 3.84743
[172]	valid_0's rmse: 3.84621
[173]	valid_0's rmse: 3.84484
[174]	valid

[114]	valid_0's rmse: 3.81074
[115]	valid_0's rmse: 3.80944
[116]	valid_0's rmse: 3.808
[117]	valid_0's rmse: 3.80308
[118]	valid_0's rmse: 3.80187
[119]	valid_0's rmse: 3.7998
[120]	valid_0's rmse: 3.79814
[121]	valid_0's rmse: 3.79632
[122]	valid_0's rmse: 3.79293
[123]	valid_0's rmse: 3.7921
[124]	valid_0's rmse: 3.79071
[125]	valid_0's rmse: 3.78919
[126]	valid_0's rmse: 3.78727
[127]	valid_0's rmse: 3.78586
[128]	valid_0's rmse: 3.78404
[129]	valid_0's rmse: 3.78307
[130]	valid_0's rmse: 3.78216
[131]	valid_0's rmse: 3.78144
[132]	valid_0's rmse: 3.77948
[133]	valid_0's rmse: 3.77705
[134]	valid_0's rmse: 3.77595
[135]	valid_0's rmse: 3.77463
[136]	valid_0's rmse: 3.7737
[137]	valid_0's rmse: 3.77254
[138]	valid_0's rmse: 3.77175
[139]	valid_0's rmse: 3.77093
[140]	valid_0's rmse: 3.77091
[141]	valid_0's rmse: 3.77044
[142]	valid_0's rmse: 3.76959
[143]	valid_0's rmse: 3.76883
[144]	valid_0's rmse: 3.76817
[145]	valid_0's rmse: 3.76749
[146]	valid_0's rmse: 3.76472
[147]	valid_0's

[87]	valid_0's rmse: 3.90007
[88]	valid_0's rmse: 3.89696
[89]	valid_0's rmse: 3.89441
[90]	valid_0's rmse: 3.89179
[91]	valid_0's rmse: 3.88853
[92]	valid_0's rmse: 3.88588
[93]	valid_0's rmse: 3.88448
[94]	valid_0's rmse: 3.88345
[95]	valid_0's rmse: 3.88257
[96]	valid_0's rmse: 3.88174
[97]	valid_0's rmse: 3.87972
[98]	valid_0's rmse: 3.87872
[99]	valid_0's rmse: 3.87303
[100]	valid_0's rmse: 3.87211
[101]	valid_0's rmse: 3.87079
[102]	valid_0's rmse: 3.87
[103]	valid_0's rmse: 3.86816
[104]	valid_0's rmse: 3.86523
[105]	valid_0's rmse: 3.86225
[106]	valid_0's rmse: 3.86065
[107]	valid_0's rmse: 3.85906
[108]	valid_0's rmse: 3.85839
[109]	valid_0's rmse: 3.85669
[110]	valid_0's rmse: 3.85478
[111]	valid_0's rmse: 3.85129
[112]	valid_0's rmse: 3.85044
[113]	valid_0's rmse: 3.84929
[114]	valid_0's rmse: 3.84845
[115]	valid_0's rmse: 3.8467
[116]	valid_0's rmse: 3.8463
[117]	valid_0's rmse: 3.84406
[118]	valid_0's rmse: 3.84343
[119]	valid_0's rmse: 3.84262
[120]	valid_0's rmse: 3.8424

[59]	valid_0's rmse: 4.02742
[60]	valid_0's rmse: 4.02007
[61]	valid_0's rmse: 4.01038
[62]	valid_0's rmse: 4.00656
[63]	valid_0's rmse: 4.00251
[64]	valid_0's rmse: 3.99937
[65]	valid_0's rmse: 3.99819
[66]	valid_0's rmse: 3.99492
[67]	valid_0's rmse: 3.99074
[68]	valid_0's rmse: 3.9892
[69]	valid_0's rmse: 3.988
[70]	valid_0's rmse: 3.98671
[71]	valid_0's rmse: 3.98457
[72]	valid_0's rmse: 3.98165
[73]	valid_0's rmse: 3.97545
[74]	valid_0's rmse: 3.97282
[75]	valid_0's rmse: 3.97188
[76]	valid_0's rmse: 3.97118
[77]	valid_0's rmse: 3.96887
[78]	valid_0's rmse: 3.96587
[79]	valid_0's rmse: 3.96292
[80]	valid_0's rmse: 3.96042
[81]	valid_0's rmse: 3.9566
[82]	valid_0's rmse: 3.95592
[83]	valid_0's rmse: 3.95477
[84]	valid_0's rmse: 3.9536
[85]	valid_0's rmse: 3.95164
[86]	valid_0's rmse: 3.95056
[87]	valid_0's rmse: 3.94855
[88]	valid_0's rmse: 3.94725
[89]	valid_0's rmse: 3.94267
[90]	valid_0's rmse: 3.94121
[91]	valid_0's rmse: 3.93346
[92]	valid_0's rmse: 3.9326
[93]	valid_0's rmse:

[31]	valid_0's rmse: 4.11751
[32]	valid_0's rmse: 4.10494
[33]	valid_0's rmse: 4.09822
[34]	valid_0's rmse: 4.09549
[35]	valid_0's rmse: 4.08686
[36]	valid_0's rmse: 4.08368
[37]	valid_0's rmse: 4.08004
[38]	valid_0's rmse: 4.0755
[39]	valid_0's rmse: 4.07221
[40]	valid_0's rmse: 4.06895
[41]	valid_0's rmse: 4.06662
[42]	valid_0's rmse: 4.06448
[43]	valid_0's rmse: 4.06291
[44]	valid_0's rmse: 4.04411
[45]	valid_0's rmse: 4.03786
[46]	valid_0's rmse: 4.02211
[47]	valid_0's rmse: 4.01784
[48]	valid_0's rmse: 4.01439
[49]	valid_0's rmse: 4.01079
[50]	valid_0's rmse: 4.0067
[51]	valid_0's rmse: 4.00344
[52]	valid_0's rmse: 3.9987
[53]	valid_0's rmse: 3.99721
[54]	valid_0's rmse: 3.99445
[55]	valid_0's rmse: 3.99132
[56]	valid_0's rmse: 3.98906
[57]	valid_0's rmse: 3.98597
[58]	valid_0's rmse: 3.98379
[59]	valid_0's rmse: 3.98347
[60]	valid_0's rmse: 3.98136
[61]	valid_0's rmse: 3.97875
[62]	valid_0's rmse: 3.97639
[63]	valid_0's rmse: 3.97437
[64]	valid_0's rmse: 3.97123
[65]	valid_0's rm

In [27]:
gbm = lgb.Booster(model_file='model_kfold_v4.txt')

In [23]:
keys = test_df['key'] 
test_df.drop(columns = ['key'], inplace = True)
test_df.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance,JFK_distance,invalid
0,-73.972484,40.742743,-73.918937,40.764496,1,2009.0,1,4,0,5.991612,16.117319,0
1,-73.985850,40.722826,-73.986301,40.739347,1,2009.0,1,4,0,0.509264,23.237424,0
2,-73.988917,40.740142,-73.982769,40.777291,1,2009.0,1,4,0,1.328903,23.142153,0
3,-73.977163,40.764490,-73.914474,40.771575,1,2009.0,1,4,0,6.974103,15.690509,0
4,-73.948849,40.778003,-73.977678,40.748692,2,2009.0,1,4,0,3.329684,19.450750,0


In [28]:
pred_fares = gbm.predict(test_df, num_iteration=gbm.best_iteration)
df_final = pd.DataFrame({'key':keys, 'fare_amount':pred_fares})

In [29]:
df_final.to_csv(r'predictions/lightgbm_withfeaures_v5.csv', index = False)